# Задача

Школа `N` имеет сильный состав
для соревнования в прыжках в длину.
В ней есть несколько сильных спортсменов,
но на соревнования нужно отправить одного.
Тренер Максим вычитал из книги, 
что длина прыжка имеет нормальное распределение с дисперсией $100$,
поэтому тренер решил выбрать лучшего школьника
на основании оценки матожидания длины прыжка.
Помогите Максиму составить симметричный
доверительный интервал этой величины
для каждого студента.

Статистическая модель:
$$
\mathcal{X} = \mathbb{R}^n,
\quad \mathcal{P} 
= \left\{\mathbb{P}_{\mu}^n 
\mid \mu \in \mathbb{R}\right\},
$$
где $\mathbb{P}_{\mu}$ - вероятностная мера
нормального распределения со средним $\mu$ и дисперсией $100$.
Тогда
$$
(X_1, \ldots, X_n) = X \in \mathcal{X}
$$
выборка из н.о.р. величин $\mathcal{N}(\mu, 100)$.
Цель - найти параметр $\mu$.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from scipy.stats import norm
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot as plt

## Точный доверительный интервал

Как было доказано на лекции, можно использовать точный доверительный интервал
$$
\mu \in I = \left(\overline{X} + z_{\alpha/2} \frac{10}{\sqrt{n}},
\overline{X} + z_{1-\alpha/2} \frac{10}{\sqrt{n}}\right).
$$

In [ ]:
def exact_solution(p: float, x: np.array) -> float:
    alpha = 1 - p
    return x.mean() + 10 * norm.ppf(alpha / 2) / np.sqrt(len(x)), \
           x.mean() + 10 * norm.ppf(1 - alpha / 2) / np.sqrt(len(x))

In [ ]:
solution = exact_solution

mu = 250
sigma = 10
p = 0.95

iter_size = 100
sample_size_list = [
    10, 
    100,
    1000
]
color_map = {
    10: "red",
    100: "blue",
    1000: "green"
}

fig = go.Figure()
fig.add_trace(go.Scatter(x=[1, iter_size], 
                         y=[mu, mu], 
                         name="Заданное значение",
                         line={"color": "black"}))

for sample_size in sample_size_list:

    x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, sample_size])
    left_side_list = []
    right_side_list = []

    for i in range(iter_size):
        left_side, right_side = solution(p, x[i])
        left_side_list.append(left_side)
        right_side_list.append(right_side)

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size), 
                             y=left_side_list, 
                             name=f"Нижняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size), 
                             y=right_side_list, 
                             name=f"Верхняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

fig.update_layout(title="Проверка доверительного интервала",
                  xaxis_title="Номер итерации",
                  yaxis_title="Значение параметра")
fig.show()

## Асимптотический доверительный интервал

Как было доказано на лекции, можно использовать 
асимптотический доверительный интервал
$$
\mu \in I = \left(\overline{X} + z_{\alpha/2} \frac{S_X}{\sqrt{n}},
\overline{X} + z_{1-\alpha/2} \frac{S_X}{\sqrt{n}}\right).
$$

In [ ]:
def clt_solution(p: float, x: np.array) -> float:
    alpha = 1 - p
    return x.mean() - np.sqrt(np.var(x)) * norm.ppf(1 - alpha / 2) / np.sqrt(len(x)), \
           x.mean() - np.sqrt(np.var(x)) * norm.ppf(alpha / 2) / np.sqrt(len(x))

In [ ]:
solution = clt_solution

mu = 250
sigma = 10
p = 0.95

iter_size = 100
sample_size_list = [
    10, 
    100,
    1000
]
color_map = {
    10: "red",
    100: "blue",
    1000: "green"
}

fig = go.Figure()
fig.add_trace(go.Scatter(x=[1, iter_size], 
                         y=[mu, mu], 
                         name="Заданное значение",
                         line={"color": "black"}))

for sample_size in sample_size_list:

    x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, sample_size])
    left_side_list = []
    right_side_list = []

    for i in range(iter_size):
        left_side, right_side = solution(p, x[i])
        left_side_list.append(left_side)
        right_side_list.append(right_side)

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size), 
                             y=left_side_list, 
                             name=f"Нижняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size), 
                             y=right_side_list, 
                             name=f"Верхняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

fig.update_layout(title="Проверка доверительного интервала",
                  xaxis_title="Номер итерации",
                  yaxis_title="Значение параметра")
fig.show()

## Сравнение методик

In [ ]:
mu = 250
sigma = 10
p = 0.95

iter_size = 10000
test_stat = [{
    "sample_size": 3
}, {
    "sample_size": 10
}, {
    "sample_size": 100
}, {
    "sample_size": 1000
}]

solution_list = [{
    "name": "clt",
    "function": clt_solution
}, {
    "name": "exact",
    "function": exact_solution
}]

for test_element in test_stat:
    x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, test_element["sample_size"]])

    for i in range(iter_size):
        for solution_element in solution_list:
            left_side, right_side = solution_element["function"](p, x[i])

            total_error_col = solution_element["name"] + "_total_error"
            total_interval_length_col = solution_element["name"] + "_total_interval_length"

            test_element[total_error_col] = test_element.get(total_error_col, 0) \
                                            + (1 if (mu < left_side or right_side < mu) else 0)
            test_element[total_interval_length_col] = test_element.get(total_interval_length_col, 0) \
                                                      + right_side - left_side

    for solution_element in solution_list:
        total_error_col = solution_element["name"] + "_total_error"
        mean_error_col = solution_element["name"] + "_mean_error"
        test_element[mean_error_col] = test_element[total_error_col] / iter_size

        total_interval_length_col = solution_element["name"] + "_total_interval_length"
        mean_interval_length_col = solution_element["name"] + "_mean_interval_length"
        test_element[mean_interval_length_col] = test_element[total_interval_length_col] / iter_size

column_description = [{
    "column": "sample_size",
    "description": "Размер выборки"
}, {
    "column": "exact_mean_error",
    "description": "Частота ошибок точного интервала"
}, {
    "column": "clt_mean_error",
    "description": "Частота ошибок асимптотического интервала"
}, {
    "column": "exact_mean_interval_length",
    "description": "Средняя длина точного интервала"
}, {
    "column": "clt_mean_interval_length",
    "description": "Средняя длина асимптотического интервала"
}]

test_data = pd.DataFrame(test_stat)
test_data[[el["column"] for el in column_description]] \
         .rename(columns={el["column"]: el["description"]
                          for el in column_description})

,Размер выборки,Частота ошибок точного интервала,Частота ошибок асимптотического интервала,Средняя длина точного интервала,Средняя длина асимптотического интервала
0,3,0.0510,0.2506,22.631715,16.403668
1,10,0.0459,0.0933,12.395901,11.388980
2,100,0.0515,0.0546,3.919928,3.889391
3,1000,0.0469,0.0470,1.239590,1.238730


## Принятие решений

В силу методики асимптотических интервалов
$$
\mathbb{P}\left(p 
\geq \overline{X} - z_{\alpha} \frac{S_X}{\sqrt{n}}\right)
\approx \alpha,
$$
где $z_{\alpha}$ - квантиль стандартного нормального распределения.
Так как мы хотим, чтобы $p$ было больше $p_0 = 0.2$,
то правая граница интервала должна быть не меньше $p_0$.
В крайнем случае
$$
p_0 = \overline{X} - z_{\alpha} \frac{S_X}{\sqrt{n}}.
$$
Отсюда
$$
z_{\alpha} = \sqrt{n} \frac{\overline{X} - p_0}{S_X}.
$$
Применяя соотношение $S_X^2 = \overline{X} (1 - \overline{X})$
и $\Phi(z_{\alpha}) = \alpha$ имеем
$$
\alpha = \Phi\left(\sqrt{n} \frac{\overline{X} - p_0}{
  \sqrt{\overline{X} (1 - \overline{X})}}\right).
$$

In [ ]:
def get_confidence(success_cnt, sample_size, threshold):
    success_freq = success_cnt / sample_size
    return norm.cdf(np.sqrt(sample_size) 
                    * (success_freq - threshold)
                    / np.sqrt(success_freq * (1 - success_freq)))

threshold = 0.22
confidence_sample1 = get_confidence(30, 100, threshold)
confidence_sample2 = get_confidence(250, 1000, threshold)
print(f"Для первой выборки уровень доверия равен {confidence_sample1}, "
      + f"а для второй - {confidence_sample2}")

Для первой выборки уровень доверия равен 0.9595722008149739, а для второй - 0.9857701315418447
